In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import *

In [34]:
MISSING = -9999

In [35]:
df = pd.read_csv("../data/processed/data_encoded.csv")

In [36]:
df = df[df["test"] == 0].drop(columns=["test"])

In [37]:
# df = df.iloc[:13000]

In [38]:
# df = df.iloc[13000:]

In [39]:
df["satisfied"].value_counts()

1.0    15626
0.0    14454
Name: satisfied, dtype: int64

In [40]:
X = df.drop(columns=["id", "satisfied"]).values
y = df["satisfied"].values

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1001)

In [42]:
# model = xgb.XGBClassifier(learning_rate=0.1, 
#                           n_estimators=600, 
#                           objective='binary:logistic',
#                           silent=True, 
#                           missing=MISSING,
#                           nthread=1)

In [43]:
# model

In [44]:
# params = {
#     'max_depth': [2, 3, 4, 5]
# #     'subsample': [0.25, 0.5, 1.],
# #     'colsample_bytree': [0.5, 1.],
# #     'colsample_bylevel': [0.5, 1.],
# #     'colsample_bynode': [0.5, 1.],
# #     'scale_pos_weight': [0.5, 1., 1.5],
# }

In [45]:
# folds = 5
# param_comb = 5

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(model, 
#                                    param_distributions=params, 
#                                    n_iter=param_comb, 
#                                    scoring='roc_auc', 
#                                    n_jobs=5, 
#                                    cv=skf.split(X_train,y_train), 
#                                    verbose=3, 
#                                    random_state=1001 )

In [46]:
# random_search.fit(X_train, y_train)

In [47]:
# pd.DataFrame(random_search.cv_results_)

In [48]:
# final_model = random_search.best_estimator_

In [49]:
# y_pred = final_model.predict(X_test)
# y_prob = final_model.predict_proba(X_test)

In [50]:
params = {"max_depth": 2}

In [51]:
# params = {
#     'subsample': 1.0,
#     'scale_pos_weight': 0.5,
#     'max_depth': 2,
#     'colsample_bytree': 1.0,
#     'colsample_bynode': 0.5,
#     'colsample_bylevel': 1.0
# }

In [52]:
final_model = xgb.XGBClassifier(
    learning_rate=0.1,
    objective='binary:logistic',
    silent=True, 
    missing=MISSING,
    nthread=8,
    **params
    )

In [53]:
final_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=-9999, n_estimators=100, n_jobs=1,
              nthread=8, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=1, verbosity=1)

In [54]:
y_pred_train = final_model.predict(X_train)
y_prob_train = final_model.predict_proba(X_train)[:,1]

In [55]:
y_pred = final_model.predict(X_test)
y_prob = final_model.predict_proba(X_test)[:,1]

In [56]:
roc_auc_score(y_test, y_prob), roc_auc_score(y_train, y_prob_train)

(0.8840862733733121, 0.8959678766510695)

In [27]:
0.8839356100336466 # type of missing using one-hot + indicator of missing

0.8839356100336466

In [26]:
0.8839356100336466 # categorical variable

0.8839356100336466

In [25]:
0.8833639755978565 # baseline (fill missing)

0.8833639755978565

In [28]:
pd.Series(final_model.feature_importances_, index=df.drop(columns=["id", "satisfied"]).columns).sort_values(ascending=False)

v98       0.280291
v79       0.103738
v101      0.048014
v224      0.042415
v223      0.029774
            ...   
v78_PS    0.000000
v78_PT    0.000000
v78_PY    0.000000
v78_RE    0.000000
v1        0.000000
Length: 1425, dtype: float32